# HCI Production Schedule

# Python Packages

In [75]:
# Install one-time for non generic packages
!pip install --user ortools

In [0]:
# 
from __future__ import print_function

# 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# 
from oauth2client.client import GoogleCredentials

# Google Collab modules
from google.colab import drive as drive_colab
from google.colab import files
from google.colab import auth

# ORTools module for optimization model

import ortools
from ortools.linear_solver import pywraplp as glp
from ortools.sat.python import cp_model

# Generic python modules for data wrangling
import numpy as np
import pandas as pd
import re

# Data Setup
Current Status: Spreadsheet Mode  

In [77]:
#@title Input Parameters

SQL_mode = False #@param {type:"boolean"}
if SQL_mode:
  print("Status: SQL Mode (not yet implemneted)")
else:
   print("Status: Spreadsheet Mode")

forecast_ready = True #@param {type:"boolean"}
if forecast_ready:
  print("Status: Using forecasting model output")
else:
   print("Status: Forecast model output not ready")

num_days =  365 #@param {type:"number"}

if forecast_ready:
  forecast_filename = 'concat_forecast.xlsx' #@param ['lower_forecast.xlsx', 'upper_forecast.xlsx', 'forecast.xlsx', 'concat_forecast.xlsx']
else:
  forecast_filename = "sales.xlsx"
  print("WARNING: Running pod = 3 and using pod3.xlsx as demand file")

pod = "3" #@param [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Status: Spreadsheet Mode
Status: Using forecasting model output


## Authentication
Based on SQL_mode Query DB using SQL or read spreadsheets from Drive.  
2 step authentication.  
*Please click on the link and navigate to get the login code.  
This will be done twice.*

In [78]:
if SQL_mode == True:
  print("SQL Mode not ready!")
else:
  # Authenticate user
  ## You must go to the link and copy access code 
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Load all the required files
  file_id = '1R2V8snJTRfHIQlYM_KBZGTBbrzUPUPy7'
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile('data.xlsx')

  file_id = '1Y8wjabOKvWDTAKwS1EyzCtL_7bJtnTWB'
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile('pod3.xlsx')

  file_id = '1oX54fAHEW7dHTTeatefM6XN3ge6kuI3T'
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile('hci_facility.xlsx')

  file_id = '1XScdiefY-mdCl4c0_511D3J_NnUF45Pq'
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile('hci_facility_test.xlsx')

  if re.search("lower", forecast_filename):
    file_id = '1grGoB9Zxz--KJPDyV78M10V_eeY0qykk'
    print("Using Lower Forecast")
  elif re.search("upper", forecast_filename): 
    file_id = '1IzWBMSTJzHkoEwhh_xGUvKg5SnmXDSVR'
    print("Using Upper Forecast")
  elif re.search("pod", forecast_filename):
    file_id = '1Y8wjabOKvWDTAKwS1EyzCtL_7bJtnTWB'
  elif re.search("concat", forecast_filename):
    file_id = '1H786zD8rg1ivtDRHahJ93U3tgfYH0tLE'
  else:
    file_id = '1Imfm7BgEvbCiLdEOrPLFMWZ3u9BZa8W_'
    print("Using Point Forecast")
  
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(forecast_filename)

  # Mount the drive to access files
  drive_colab.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Query Data

In [13]:
if SQL_mode == True:
  print("SQL Mode not ready!")
  # INSERT INTO #temp
  # SELECT	F.ID AS id,
  # 		F.DivisionNumber AS div_number,
  # 		F.DivisionName AS div_name,
  # 		F.DivisionType AS div_type_old,
  # 		F.OnSiteProductionCapLbs AS production,
  # 		F.Pod AS pod,
  # 		(F.OnSiteStorageCapSkids+F.FleetPalletCount) AS storage,
  # 		CASE WHEN F.OnSiteProductionCapLbs > 0 THEN 'PLANT'
  # 			ELSE 'TERM' END AS div_type_new
  # FROM HcFacilities AS F
  # WHERE pod IS NOT NULL
  # ORDER BY div_type_new,div_number;
  # 
  # SELECT *, ROW_NUMBER() OVER(PARTITION BY div_type_new ORDER BY div_number) AS 'index'
  # FROM #temp
  # WHERE pod = pod
  # 
  # DELETE #temp

else:
  # Create dataframes
  ## Beautiful demands with nice numbers

  ## Realistic demands
  #plant_demand = pd.read_excel("pod3.xlsx",sheet_name="plant_demand_int")
  #terminal_demand = pd.read_excel("pod3.xlsx",sheet_name="term_demand_int")
  
  # Both demands populated from same file
  plant_demand = pd.read_excel(forecast_filename,sheet_name="Sheet1",index_col=0)
  terminal_demand = pd.read_excel(forecast_filename,sheet_name="Sheet1",index_col=0)

  # plant_capacity = pd.read_excel("data.xlsx",sheet_name="O",header=None)
  # plant_storage = pd.read_excel("data.xlsx",sheet_name="m",header=None)
  # terminal_storage = pd.read_excel("data.xlsx",sheet_name="n",header=None)
  # plant_demand=pd.read_excel("data.xlsx",sheet_name="act_plant_demand",header=None)
  # terminal_demand=pd.read_excel("data.xlsx",sheet_name="act_term_demand",header=None)

  ## Distance matrix
  dist_matrix = pd.read_excel("hci_facility_test.xlsx",sheet_name="miles",index_col=0)


  ## Facility details
  facility_df = pd.read_excel("hci_facility_test.xlsx",sheet_name="facility_list", dtype={'div_number':str})
  plant_map = facility_df[facility_df['div_type_new']=="PLANT"][['div_number','index']]
  plant_map = plant_map.set_index('index')
  term_map = facility_df[facility_df['div_type_new']=="TERM"][['div_number','index']]
  term_map = term_map.set_index('index')

  print("Loaded spreadsheets")


Loaded spreadsheets


In [14]:
plant_demand.head()

,Date,010,011,020,021,022,030,040,041,050,051,054,060,062,070,080,082,090,091,100,101,110,111,113,116,120,124,130,131,140,141,150,151,154,156,160,162,163,164,170,...,284,290,291,400,401,410,420,421,423,430,440,441,450,451,452,453,460,461,470,471,480,490,491,492,500,501,510,520,530,532,540,550,551,552,560,570,580,581,590,670
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2019-11-21 00:00:00,31.694505,4.077872,49.680353,0.018078,1.934355,44.329330,43.881289,8.561430,45.541977,10.267685,5.805535,24.352237,2.549591,51.785319,45.938185,2.871374,32.161009,3.324849,42.042519,11.761371,28.710996,1.722816,3.423180,4.213395,46.352213,2.956188,37.002027,9.411799,31.214507,8.228457,45.752951,12.746191,6.341194,6.067953,53.991441,4.556285,4.731355,5.085701,25.282114,...,6.254848,3.925082,2.332331,20.824141,8.721967,14.601289,44.829398,10.397017,5.602036,60.214090,43.438370,7.480097,31.869297,9.510384,3.143993,1.949539,13.136972,2.031340,58.957274,15.816221,59.041094,31.231366,11.194547,3.193179,8.826348,0.809304,15.617772,22.887153,33.766554,7.978199,13.707488,28.611649,6.822811,6.341457,27.989759,25.822471,41.787033,7.072054,11.803086,13.755694
2,2019-11-22 00:00:00,51.301672,4.810036,80.414103,0.029261,2.564970,62.180366,57.300187,11.179515,51.579540,11.628886,6.575183,33.012559,3.456295,67.621270,54.186182,3.386916,41.995845,4.426539,54.899121,16.497573,46.218230,2.773345,5.510548,6.782616,50.831527,3.489359,41.907437,10.659534,36.844277,9.712521,54.004838,15.045062,7.484876,7.162353,55.382175,4.673648,4.853227,5.216700,45.115318,...,10.090662,11.153508,6.627550,22.560166,9.449082,19.793908,48.566644,11.263775,6.069055,64.422789,46.474521,8.002923,51.302383,15.309574,5.061120,3.138318,23.442608,3.624877,60.475920,16.223622,69.160036,35.371752,12.678624,3.616504,9.443271,0.865871,18.434554,25.921335,39.553740,9.345567,24.460680,32.264672,7.693920,7.151109,36.549027,20.264564,54.565507,9.587070,12.943694,16.113256
3,2019-11-23 00:00:00,43.883213,4.044191,68.785852,0.025030,2.006428,49.352319,48.313048,9.426085,39.555308,8.917958,5.042375,38.183194,3.997642,57.015339,45.558762,2.847658,35.409086,4.101313,46.288572,13.094060,45.320818,2.719495,5.403550,6.650919,29.766821,1.889573,32.137968,8.174582,19.952073,5.259567,29.244935,8.147268,4.053243,3.878589,34.933835,2.948032,3.061307,3.290578,41.455884,...,3.921368,9.125547,5.422511,18.376613,7.696846,22.894154,39.560454,9.175022,4.943611,58.881855,42.477298,7.314600,50.306253,15.012311,4.962849,3.077381,21.541110,3.330853,38.146855,10.233497,45.433926,27.125882,9.722980,2.773424,8.631066,0.791398,9.982760,19.878548,25.984395,6.139468,22.476603,23.200986,5.532569,5.142243,30.816564,0.000000,46.007283,11.088657,7.579797,10.585427
4,2019-11-24 00:00:00,18.262656,1.283225,28.626262,0.010417,0.864504,12.532504,30.747087,5.998890,20.287699,4.573972,2.586206,8.527561,0.892805,36.285345,14.455824,0.903564,22.534829,1.130261,29.458683,3.325099,5.726550,0.343624,0.682770,0.840382,15.815952,0.099217,16.483386,4.192698,1.047639,0.276168,1.535587,0.427795,0.212827,0.203656,9.365223,0.790322,0.820689,0.882153,9.096181,...,0.000000,0.585991,0.348203,10.188659,4.267410,5.113017,21.933748,5.086964,2.740917,31.131310,22.458089,3.867288,6.356488,1.896893,0.627085,0.388845,4.726514,0.730850,10.226584,2.743443,11.169291,13.912716,4.986863,1.422474,4.56

In [0]:
# Defaults
max_shifts = 3                  # Integer: Number of shifts we can have at plant
num_oss = 1                     # Integer: # OSSs in the system
init_storage_factor = 0         # Real(0,1): Initial storage (Plant & Terminal)
min_storage_factor = 0          # Real(0,1): Minimum storage required (Plant & Terminal)
print_days = 3                  # Integer: Number of days to print
print_prod = 1                  # Binary: Turn on production verbose
print_inventory = 1             # Binary: Turn on inventory verbose
print_cust = 0                  # Binary: Turn on customer verbose
print_transport = 0             # Binary: Turn on transportation verbose
d = terminal_demand             # Array: Demand (Terminal)
b = plant_demand                # Array: Demand (Plant)
#TrCap = 20
max_dist = dist_matrix.max().max()

if SQL_mode:
  print("SQL Mode not ready!")
else:
  df_model = facility_df[facility_df["pod"]==int(pod)]
  df_model_plant = df_model[df_model["div_type_new"]=='PLANT']
  df_model_term = df_model[df_model["div_type_new"]=='TERM']
  num_plants = df_model_plant.shape[0]
  num_terms = df_model_term.shape[0]
  O = list(round(df_model_plant["production"]/max_shifts,0))
  M = list(df_model_plant["storage"])
  N = list(df_model_term["storage"])
  T = 1
  C = [83]*num_plants          # Array: Production cost
  F = list(df_model_plant["Semi"])  # Fleet capacity - Semi trucks

In [16]:
# Sets
num_days_pad = num_days+1
plants = list(range(1,num_plants+1))			#! Set of Plants (49) = I
terminals = list(range(1,num_terms+1))			#! Set of Terminals (55) = J
oss = list(range(1,num_oss+1))
days = list(range(1,num_days_pad+1+1))		#! Set of Days (365) = K
A = list(range(2,num_days_pad+1))
print(days)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

# Pre-Processing
Check that production can match demand for each of the days forecasted

In [17]:
# Cumulative Production should match Cumulative Demand on daily basis
for day in days:
    lhs = sum(O)*(day+1)*max_shifts
    rhs = 0
    for fac in df_model['div_number']:
      rhs = rhs + b.loc[0:day+1,fac].sum()
    # rhs = d.iloc[0:day+1,:].sum().sum() + b.iloc[0:day+1,:].sum().sum()
    if(lhs < rhs):
        print("Production (%s) cannot match Demand (%s) cumulative on Day %s" % (lhs,rhs,day))
        break;
    if(day == num_days_pad+1):
        print("Total Production is %s > Total Demand is %s over %s Days" % (lhs, round(rhs,1), day))


Total Production is 143520.0 > Total Demand is 94762.0 over 367 Days


# Optimization Model

## Model Definition
Define all model decision variables

In [0]:
#model = cp_model.CpModel()
# initialize LP model object
model = glp.Solver('HCI', glp.Solver.GLOP_LINEAR_PROGRAMMING)

P = {}
for plant in plants:
    for day in days:
        P[(plant, day)] = model.NumVar(0,100000,'P_%i,%i' % (plant, day))
X = {}
for plant in plants:
    for terminal in terminals:
        for day in days:
            X[(plant,terminal, day)] = model.NumVar(0,100000,'X_%i,%i,%i' % (plant,terminal, day))
Z = {}
for plant in plants:
    for day in days:
        Z[(plant, day)] = model.NumVar(0,3,'Z_%i,%i' % (plant, day))
Y = {}
for plant in plants:
    for day in days:
        Y[(plant, day)] = model.NumVar(0,100000,'Y_%i,%i' % (plant, day))
s = {}
for storage in oss:
    for day in days:
        s[(storage, day)] = model.NumVar(0,100000,'s_%i,%i' % (storage, day))
PS = {}
for plant in plants:
    for storage in oss:
        for day in days:
            PS[(plant,storage,day)] = model.NumVar(0,100000,'PS_%i,%i,%i' % (plant,storage,day))
SP = {}
for storage in oss:
    for plant in plants:
        for day in days:
            SP[(storage,plant,day)] = model.NumVar(0,100000,'SP_%i,%i,%i' % (storage,plant,day))
ST = {}
for storage in oss:
    for terminal in terminals:
        for day in days:
            ST[(storage,terminal,day)] = model.NumVar(0,100000,'ST_%i,%i,%i' % (storage,terminal,day))
m = {}
for plant in plants:
    for day in days:
        m[(plant, day)] = model.NumVar(0,100000,'m_%i,%i' % (plant, day))
B = {}
for plant in plants:
    for day in days:
        B[(plant, day)] = model.NumVar(0,100000,'B_%i,%i' % (plant, day))
n = {}
for terminal in terminals:
    for day in days:
        n[(terminal, day)] = model.NumVar(0,100000,'n_%i,%i' % (terminal, day))
D = {}
for terminal in terminals:
    for day in days:
        D[(terminal, day)] = model.NumVar(0,100000,'D_%i,%i' % (terminal, day))
# Arcs from Plant storage to Terminal
W = {}
for plant in plants:
    for terminal in terminals:
        for day in days:
            W[(plant,terminal, day)] = model.NumVar(0,100000,'W_%i,%i,%i' % (plant,terminal, day))
# Fleet deficit
Tr= {}
for plant in plants:
  for day in days:
    Tr[(plant,day)] = model.NumVar(0,100000,'Tr_%i,%i' % (plant,day))


## Constraints
Add bounding equations on decision variables

In [0]:
### Plant production capacity
for plant in plants:
    for day in days:
        model.Add(P[(plant, day)] == Z[(plant, day)] * O[plant-1])
        # plant_cap[plant][day] = mymodel.Constraint(0, Z[(plant, day)]*O[plant-1])

In [0]:
### Produce = Sent + Inv
for plant in plants:
    for day in A:
        model.Add(P[(plant, day)] == Y[(plant, day)]
                                     + sum(PS[(plant,storage,day)] for storage in oss )
                                     + sum(X[(plant,terminal,day)] for terminal in terminals ))

In [0]:
### Inv equation for plants
for plant in plants:
    for day in A:
        fac_plant = df_model_plant.loc[df_model_plant['index'] == plant,'div_number']
        model.Add(m[(plant, day)] == m[(plant, day-1)]
                                      - b[fac_plant.iloc[0]][day-1]
                                      + sum(SP[(storage,plant,day)] for storage in oss)
                                      + Y[(plant, day)]
                                      - sum(W[(plant,terminal,day)] for terminal in terminals)
                 )

In [0]:
### Inv equation for terminals
for terminal in terminals:
    for day in A:
        fac_term = df_model_term.loc[df_model_term['index'] == terminal,'div_number']
        model.Add(n[(terminal, day)] == n[(terminal, day-1)]
                                        - d[fac_term.iloc[0]][day-1]
                                        + sum(ST[(storage,terminal,day)] for storage in oss)
                                        + sum(X[(plant,terminal,day)] for plant in plants)
                                        + sum(W[(plant,terminal,day)] for plant in plants)
                 )

In [0]:
### Inv equation for OSS
for storage in oss:
    for day in A:
        model.Add(s[(storage,day)] == s[(storage,day-1)]
                                      + sum(PS[(plant,storage,day)] for plant in plants)
                                      - sum(ST[(storage,terminal,day)] for terminal in terminals)
                                      - sum(SP[(storage,plant,day)] for plant in plants))

In [0]:
### Ensure fleet capacity
for plant in plants:
    for day in days:
        model.Add(sum((X[(plant,terminal,day)]+W[(plant,terminal,day)]) for terminal in terminals) + sum(PS[(plant,storage,day)] for storage in oss )<= 25 * (F[plant-1] + Tr[(plant,day)]))  
        #model.Add(sum((X[(plant,terminal,day)]+W[(plant,terminal,day)]) for terminal in terminals) <= 25 * F[plant-1])

In [0]:
### Maximum inventory plants
for plant in plants:
    for day in A:
        model.Add(m[(plant, day)] <= M[plant-1])

In [0]:
### Maximum inventory terminals
for terminal in terminals:
    for day in A:
        model.Add(n[(terminal, day)] <= N[terminal-1])

In [0]:
### Init Values
# Plant
for plant in plants:
    model.Add(m[(plant, 1)] == init_storage_factor * M[plant-1])

for plant in plants:
    for day in days:
        model.Add(min_storage_factor * M[plant-1] <= m[(plant, day)])

for plant in plants:
    model.Add(Z[(plant, 1)] == 0)
    model.Add(Z[(plant, num_days_pad+1)] == 0)

# Terminal
for terminal in terminals:
    model.Add(n[(terminal, 1)] == init_storage_factor * N[terminal-1])

for terminal in terminals:
    for day in days:
        model.Add(min_storage_factor * N[terminal-1] <= n[(terminal, day)])

# OSS
for storage in oss:
    model.Add(s[(storage, 1)] == 0)

## Objective function
Minimize cost on production, transportation and "inventory"

add a new decision variable(extra fleet) for fleet capacity and add that to the obj fucntin with high cost

In [0]:
# define objective function
TotCost = model.Objective()
TotCost.SetMinimization()
for plant in plants:
    for day in A:
        TotCost.SetCoefficient(P[(plant,day)], C[plant-1])
for plant in plants:
    for terminal in terminals:
        for day in days:
            # Query from df_model each pair of plant, terminal
            fac1 = df_model_plant.loc[df_model_plant['index'] == plant,'div_number']
            fac2 = df_model_term[df_model_term['index'] == terminal]['div_number']
            dist = dist_matrix[fac1.iloc[0]][fac2.iloc[0]]
            TotCost.SetCoefficient(X[(plant,terminal,day)], dist)
for plant in plants:
    for terminal in terminals:
        for day in days:
            # Query from df_model each pair of plant, terminal
            fac1 = df_model_plant.loc[df_model_plant['index'] == plant,'div_number']
            fac2 = df_model_term[df_model_term['index'] == terminal]['div_number']
            dist = dist_matrix[fac1.iloc[0]][fac2.iloc[0]]
            TotCost.SetCoefficient(W[(plant,terminal,day)], dist)
for plant in plants:
    for day in days:
        TotCost.SetCoefficient(m[(plant, day)], 1)
for terminal in terminals:
    for day in days:    
        TotCost.SetCoefficient(n[(terminal, day)], 1)
for plant in plants:
    for storage in oss:
        for day in days:
            TotCost.SetCoefficient(PS[(plant,storage,day)],max_dist)
for storage in oss:
    for plant in plants:
        for day in days:          
            TotCost.SetCoefficient(SP[(storage,plant,day)],max_dist) 
for storage in oss:
    for terminal in terminals:
        for day in days:
            TotCost.SetCoefficient(ST[(storage,terminal,day)],max_dist)
for storage in oss:
    for day in days:
        TotCost.SetCoefficient(s[(storage, day)], 2)

for plant in plants:
  for day in days:
    TotCost.SetCoefficient(Tr[(plant,day)], 2)



In [29]:
# Solve the model and print optimal solution
status = model.Solve()                 # solve mymodel and display the solution
# solver = cp_model.CpSolver()
# status = solver.Solve(model)

print('Solution Status =', status)
print('Number of variables =', model.NumVariables())
print('Number of constraints =', model.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Cost = %.2f' % TotCost.Value())
print('Time= ',model.WallTime())

Solution Status = 0
Number of variables = 32296
Number of constraints = 14652
Optimal Solution:
Total Cost = 12880201.05
Time=  26586


## Output
Print decision variables

In [30]:
production1 = list()
production2 = list()
production = list()
for day in days:
    dummy = list()
    for plant in plants:
        dummy.append(P[(plant,day)].solution_value())
        if P[(plant,day)].solution_value() > 0:
          print('Day %s Plant %s' % (day, plant))
          print('%3.1f' % (P[(plant,day)].solution_value()))
        if plant == 1:
            production1.append(P[(plant,day)].solution_value())
        if plant == 2:
            production2.append(P[(plant,day)].solution_value())
    production.append(dummy)

df_prod = pd.DataFrame(production)


Day 2 Plant 1
85.4
Day 2 Plant 2
10.3
Day 2 Plant 3
46.4
Day 2 Plant 4
34.4
Day 3 Plant 1
96.8
Day 3 Plant 2
11.6
Day 3 Plant 3
52.6
Day 3 Plant 4
39.0
Day 4 Plant 1
74.2
Day 4 Plant 2
8.9
Day 4 Plant 3
40.3
Day 4 Plant 4
29.9
Day 5 Plant 1
38.1
Day 5 Plant 2
4.6
Day 5 Plant 3
20.7
Day 5 Plant 4
15.3
Day 6 Plant 1
102.8
Day 6 Plant 2
12.4
Day 6 Plant 3
55.8
Day 6 Plant 4
41.4
Day 7 Plant 1
100.1
Day 7 Plant 2
12.0
Day 7 Plant 3
54.4
Day 7 Plant 4
40.4
Day 8 Plant 1
84.5
Day 8 Plant 2
10.2
Day 8 Plant 3
45.9
Day 8 Plant 4
34.1
Day 9 Plant 1
83.7
Day 9 Plant 2
10.1
Day 9 Plant 3
45.5
Day 9 Plant 4
33.7
Day 10 Plant 1
94.0
Day 10 Plant 2
11.3
Day 10 Plant 3
51.0
Day 10 Plant 4
37.9
Day 11 Plant 1
70.4
Day 11 Plant 2
8.5
Day 11 Plant 3
38.2
Day 11 Plant 4
28.4
Day 12 Plant 1
33.3
Day 12 Plant 2
4.0
Day 12 Plant 3
18.1
Day 12 Plant 4
13.4
Day 13 Plant 1
97.1
Day 13 Plant 2
11.7
Day 13 Plant 3
52.7
Day 13 Plant 4
39.1
Day 14 Plant 1
93.7
Day 14 Plant 2
11.3
Day 14 Plant 3
50.9
Day 14 Plant 4

In [0]:
shifts = list()
for day in days:
    dummy = list()
    for plant in plants:
        dummy.append(Z[(plant,day)].solution_value())
    shifts.append(dummy)

df_shifts = pd.DataFrame(shifts)



In [0]:
plant_inv1 = list()
plant_inv2 = list()
plant_inv = list()
for plant in plants:
    dummy = list()
    for day in days:
        dummy.append(m[(plant,day)].solution_value())
        #if m[(plant,day)].solution_value() > 0:
        #   print('Inventory Day %s Plant %s' % (day, plant))
        #   print('%3.1f' % (m[(plant,day)].solution_value()))   
        if plant == 1:
           plant_inv1.append(m[(plant,day)].solution_value())
        if plant == 2:
           plant_inv2.append(m[(plant,day)].solution_value())
           #print('Inventory Day %s Plant %s' % (day, plant))
           #print('%3.1f' % (m[(plant,day)].solution_value()))
    plant_inv.append(dummy)

#df_inv = pd.DataFrame( {"Plant1":plant_inv1, "Plant2":plant_inv2})
df_plant_inv = pd.DataFrame(plant_inv)


In [0]:
term_inv1 = list()
term_inv2 = list()
term_inv3 = list()
term_inv = list()
for terminal in terminals:
    dummy = list()
    for day in days:
        dummy.append(n[(terminal,day)].solution_value())
        #if n[(terminal,day)].solution_value() > 0:
        #   print('Inventory Day %s Terminal %s' % (day, terminal))
        #   print('%3.1f' % (n[(terminal,day)].solution_value()))
        if terminal == 1:
           term_inv1.append(n[(terminal,day)].solution_value())
        if terminal == 2:
           term_inv2.append(n[(terminal,day)].solution_value())
        if terminal == 3:
           term_inv3.append(n[(terminal,day)].solution_value())
    term_inv.append(dummy)

df_term_inv = pd.DataFrame(term_inv)


In [0]:
oss_inv = list()
for storage in oss:
    for day in days:
        #if s[(storage,day)].solution_value() > 0:
        #   print('Storage Day %s OSS %s' % (day, storage))
        #   print('%3.1f' % (s[(storage,day)].solution_value()))
        oss_inv.append(s[(storage,day)].solution_value())

In [0]:
# for plant in plants:
#     for storage in oss:
#         for day in days:
#             if PS[(plant,storage,day)].solution_value() > 0:
#                 print('Plant to Storage Day %s Plant %s OSS %s' % (day, plant, storage))
#                 print('%3.1f' % (PS[(plant,storage,day)].solution_value()))

In [0]:
# for storage in oss:
#     for plant in plants:
#         for day in days:
#             if SP[(storage,plant,day)].solution_value() > 0:
#                 print('Storage to Plant Day %s Storage %s Plant %s' % (day,storage, plant))
#                 print('%3.1f' % (SP[(storage,plant,day)].solution_value()))

In [0]:
# for storage in oss:
#     for terminal in terminals:
#         for day in days:
#             if ST[(storage,terminal,day)].solution_value() > 0:
#                 print('Storage to Terminal Day %s Storage %s Terminal %s' % (day,storage,terminal))
#                 print('%3.1f' % (ST[(storage,terminal,day)].solution_value()))

In [0]:
# for day in days:
#     #daily_sum = list()
#     #daily_sum[day] = 0
#     for plant in plants:
#         for terminal in terminals:
#             if X[(plant,terminal, day)].solution_value() > 0:
#                 print('Transportation Day %s Plant %s Terminal %s' % (day, plant, terminal))
#                 print('%3.1f' % (X[(plant,terminal, day)].solution_value()))
#                 #daily_sum[day] = daily_sum[day] + X[(plant,terminal, day)].solution_value()

In [0]:
# for day in days:
#     for plant in plants:
#         for terminal in terminals:
#             if W[(plant,terminal, day)].solution_value() > 0:
#                 print('Transportation Day %s Plant %s Terminal %s' % (day, plant, terminal))
#                 print('%3.1f' % (W[(plant,terminal, day)].solution_value()))

In [40]:
for plant in plants:
  for day in days:
    if Tr[(plant,day)].solution_value() > 0:
      print('truck deficit Day %s Plant %s ' % (day, plant))
      print('%3.1f' % (Tr[(plant,day)].solution_value()))


truck deficit Day 136 Plant 1 
0.4
truck deficit Day 137 Plant 1 
0.3
truck deficit Day 165 Plant 1 
0.1
truck deficit Day 166 Plant 1 
1.2
truck deficit Day 173 Plant 1 
0.0
truck deficit Day 319 Plant 1 
0.1
truck deficit Day 320 Plant 1 
0.1
truck deficit Day 326 Plant 1 
0.2
truck deficit Day 328 Plant 1 
0.3
truck deficit Day 330 Plant 1 
0.0
truck deficit Day 331 Plant 1 
0.0
truck deficit Day 333 Plant 1 
0.0
truck deficit Day 334 Plant 1 
0.3
truck deficit Day 337 Plant 1 
0.1
truck deficit Day 338 Plant 1 
0.3
truck deficit Day 339 Plant 1 
0.1
truck deficit Day 342 Plant 1 
0.1
truck deficit Day 343 Plant 1 
0.2
truck deficit Day 349 Plant 1 
0.0
truck deficit Day 363 Plant 1 
0.0
truck deficit Day 152 Plant 2 
1.2
truck deficit Day 153 Plant 2 
0.9
truck deficit Day 154 Plant 2 
0.9
truck deficit Day 155 Plant 2 
1.0
truck deficit Day 156 Plant 2 
1.0
truck deficit Day 157 Plant 2 
1.0
truck deficit Day 158 Plant 2 
1.1
truck deficit Day 159 Plant 2 
1.2
truck deficit Day 16

In [41]:
df_O = pd.DataFrame(O)
df_O.head()

,0
0,39.0
1,13.0
2,39.0
3,39.0


In [42]:
df_M = pd.DataFrame(M)
df_M.head()

,0
0,290
1,105
2,165
3,220


In [43]:
df_N = pd.DataFrame(N)
df_N.head()

,0
0,110
1,110
2,81
3,35
4,645


In [0]:
# # Checking fleet capacity
# for plant in plants:
#     for day in days:
#       if sum((X[(plant,terminal,day)].solution_value()+W[(plant,terminal,day)].solution_value()) for terminal in terminals) + sum(PS[(plant,storage,day)].solution_value() for storage in oss ) > 0:
#         print('Outgoing pallets from plant %s on day %s ' % (plant, day))
#         #print('%3.1f' % (PS[(plant,storage,day)].solution_value()))
#         print('%3.1f' % (sum((X[(plant,terminal,day)].solution_value()+W[(plant,terminal,day)].solution_value()) for terminal in terminals) + sum(PS[(plant,storage,day)].solution_value() for storage in oss )))
#         #sum((X[(plant,terminal,day)]+W[(plant,terminal,day)]) for terminal in terminals) + sum(PS[(plant,storage,day)] for storage in oss )

total_inv = np.array(term_inv1) + np.array(term_inv2) + np.array(term_inv3) + np.array(plant_inv1) + np.array(plant_inv2)
total_production = np.array(production1) + np.array(production2)
total_demand = np.array(d.sum(axis=1) + b.sum(axis=1))
total_demand = np.roll(total_demand, 1)
total_min_inv = min_storage_factor*(sum(N)+sum(M))
total_oss = np.array(oss_inv)

In [0]:
# for day in days:
#   for plant in plants:
#     if plant == 2:
#       print('Day %s' % day)
#       print('Inv %3.1f' % (m[(plant,day)].solution_value()))
#       print('Prod %3.1f' % (P[(plant,day)].solution_value()))
#       print('Transport %3.1f' % (sum(X[(plant,terminal,day)].solution_value() 
#                                             + W[(plant,terminal,day)].solution_value()
#                                             for terminal in terminals )))
#       print('OSS Out%3.1f' % (PS[(plant,1,day)].solution_value()) )
#       print('OSS to Plant %3.1f' % (SP[(1, plant,day)].solution_value()) )
#       print('Demand %3.1f' % b[plant-1][day-1] )
#       print()

## Visualizations

In [0]:
# total_demand = 0
# df_model['div_number']
# for fac in df_model['div_number']:
#   print(fac)
#   #total_demand = total_demand + b[fac].sum(axis=1)
# print(total_demand)
# #total_demand = np.array(d.sum(axis=1))
# total_demand = np.roll(total_demand, 1)

total_demand = b.loc[:,df_model['div_number']].sum(axis=1)

In [0]:
total_inv = df_plant_inv.sum() + df_term_inv.sum()
total_production = df_prod.sum(axis=1)
total_oss = np.array(oss_inv)
total_min_inv = min_storage_factor*(sum(N)+sum(M))

In [48]:
df_OSS_TOT = pd.DataFrame(total_oss)
df_OSS_TOT

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
362,0.0
363,0.0
364,0.0
365,0.0


In [0]:
df_term_inv.to_excel("pod3_results.xlsx", sheet_name="term_inv")
df_plant_inv.to_excel("pod3_results.xlsx", sheet_name="plant_inv")
df_shifts.to_excel("pod3_results.xlsx", sheet_name="shifts")
df_prod.to_excel("pod3_results.xlsx", sheet_name="production")

In [50]:
#Creating X dataframe
X1 = list()
X2 = list()
X_tot = list()
for plant in plants:
  for terminal in terminals:
    dummy=list()
    for day in days:
      dummy.append(X[(plant,terminal,day)].solution_value())
      #if m[(plant,day)].solution_value() > 0:
      #   print('Inventory Day %s Plant %s' % (day, plant))
      #   print('%3.1f' % (m[(plant,day)].solution_value()))   
      if plant == 1:
        X1.append(X[(plant,terminal,day)].solution_value())
      if plant == 2:
        X2.append(X[(plant,terminal,day)].solution_value())
    X_tot.append(dummy)

#df_inv = pd.DataFrame( {"Plant1":plant_inv1, "Plant2":plant_inv2})
df_X_tot = pd.DataFrame(X_tot)
df_X_tot.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366
0,0.0,5.805535,6.575183,5.042375,2.586206,6.983157,6.805850,5.743523,5.688052,6.385406,4.782241,2.259712,6.596225,6.366141,5.260203,5.171514,5.846966,4.233804,1.713428,6.064137,5.859841,4.790503,4.748138,5.478281,3.926586,1.472676,5.892941,5.759301,4.759711,4.784213,5.576459,4.080367,1.674017,6.132491,6.026674,5.043782,5.073439,5.859204,4.345230,1.910149,...,9.791903,9.585505,8.496192,8.414439,9.084805,7.452571,4.897598,9.197124,8.924507,7.769685,7.625187,8.237578,6.554044,3.956192,8.222798,7.928508,6.764257,6.623251,7.252391,5.598845,3.043846,7.365452,7.137255,6.048836,5.991777,6.711131,5.152041,2.693593,7.111633,6.977539,5.978734,6.004757,6.798786,5.304311,2.899035,7.357721,7.250999,6.265895,6.291914,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.0,11.194547,12.678624,9.722980,4.986863,13.465302,13.123408,11.074972,10.968010,12.312685,9.221376,4.357299,12.719199,12.275539,10.143008,9.971993,11.274436,8.163850,3.303924,11.693198,11.299264,9.237307,9.155618,10.563519,7.571456,2.839694,11.363087,11.105397,9.177932,9.225179,10.752830,7.867985,3.227930,11.825001,11.620959,9.725693,9.782879,11.298034,8.378707,3.683253,...,8.021515,16.189738,16.382818,16.225178,17.517813,14.370451,9.443813,17.734392,17.208717,14.981927,14.703298,15.884145,12.637863,7.628543,15.855645,15.288179,13.043207,12.771311,13.984454,10.795996,5.869310,14.202463,13.762441,11.663693,11.553669,12.940767,9.934444,5.193931,13.713035,13.454469,11.528519,11.578697,13.109787,10.228060,5.590076,14.187556,13.981769,12.082238,12.132410,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.0,22.887153,25.921335,19.878548,10.195598,27.529692,26.830694,22.642684,22.423999,25.173177,18.853022,8.908461,26.004291,25.097231,20.737292,20.387653,23.050485,16.690920,6.754845,23.906640,23.101246,18.885593,18.718580,21.597021,15.479776,5.805729,23.231732,22.704886,18.764202,18.860797,21.984067,16.086027,6.599475,24.176111,23.758950,19.884095,20.001011,23.098731,17.130193,7.530379,...,38.602607,24.224756,25.471938,26.352651,19.130912,29.380260,43.813413,23.068484,23.866777,30.630418,34.277816,28.257943,25.838000,19.596051,28.417167,31.256547,26.666722,26.110834,28.591093,22.072319,11.999752,29.036812,28.137190,23.846317,23.621375,26.457285,20.310884,10.618947,28.036181,27.507544,23.569957,23.672545,26.802844,20.911179,11.428862,29.006335,28.585604,24.702029,24.804605,0.0


In [0]:
TOT_X=df_X_tot.transpose()

In [52]:
#Creating PS dataframe
PS1 = list()
PS2 = list()
PS_tot = list()
for plant in plants:
   for storage in oss:
    dummy = list()
    for day in days:
        dummy.append(PS[(plant,storage,day)].solution_value())
        #if m[(plant,day)].solution_value() > 0:
        #   print('Inventory Day %s Plant %s' % (day, plant))
        #   print('%3.1f' % (m[(plant,day)].solution_value()))   
        if plant == 1:
           PS1.append(PS[(plant,storage,day)].solution_value())
        if plant == 2:
           PS2.append(PS[(plant,storage,day)].solution_value())
    PS_tot.append(dummy)

#df_inv = pd.DataFrame( {"Plant1":plant_inv1, "Plant2":plant_inv2})
df_PS_tot = pd.DataFrame(PS_tot)
df_PS_tot.shape

(4, 367)

In [53]:
TOT_PS=df_PS_tot.transpose().astype(int)
TOT_PS.shape

(367, 4)

In [54]:
#Creating W dataframe
W1 = list()
W2 = list()
W_tot = list()
for plant in plants:
  for terminal in terminals:
    dummy = list()
    for day in days:
          dummy.append(W[(plant,terminal,day)].solution_value())
          #if m[(plant,day)].solution_value() > 0:
          #   print('Inventory Day %s Plant %s' % (day, plant))
          #   print('%3.1f' % (m[(plant,day)].solution_value()))   
          if plant == 1:
            W1.append(W[(plant,terminal,day)].solution_value())
          if plant == 2:
            W2.append(W[(plant,terminal,day)].solution_value())
    W_tot.append(dummy)

#df_inv = pd.DataFrame( {"Plant1":plant_inv1, "Plant2":plant_inv2})
df_W_tot = pd.DataFrame(W_tot)
df_W_tot.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
TOT_W=df_W_tot.transpose()

In [0]:
XW_add = TOT_W.add(TOT_X, fill_value=0).astype(int)

In [0]:
# Change column names to be meaningful:

plant_code = facility_df[(facility_df['div_type_new']=="PLANT") & (facility_df['pod']==int(pod))][['div_number','index','pod']].sort_values('index',ascending=True)
term_code = facility_df[(facility_df['div_type_new']=="TERM") & (facility_df['pod']==int(pod))][['div_number','index','pod']].sort_values('index',ascending=True)
setcolnames= ['%s to %s' %(p,t) for p in plant_code.div_number for t in term_code.div_number ]
XW_add.columns=setcolnames

In [58]:
XW_add.head(30).style.background_gradient(cmap='Blues')

,050 to 054,050 to 131,050 to 491,050 to 492,050 to 520,051 to 054,051 to 131,051 to 491,051 to 492,051 to 520,130 to 054,130 to 131,130 to 491,130 to 492,130 to 520,490 to 054,490 to 131,490 to 491,490 to 492,490 to 520
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,0,11,0,22,0,0,0,0,0,0,9,0,0,0,0,0,0,3,0
2,6,0,12,0,25,0,0,0,0,0,0,10,0,0,0,0,0,0,3,0
3,5,0,9,0,19,0,0,0,0,0,0,8,0,0,0,0,0,0,2,0
4,2,0,4,0,10,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0
5,6,0,13,0,27,0,0,0,0,0,0,11,0,0,0,0,0,0,3,0
6,6,0,13,0,26,0,0,0,0,0,0,11,0,0,0,0,0,0,3,0
7,5,0,11,0,22,0,0,0,0,0,0,9,0,0,0,0,0,0,3,0
8,5,0,10,0,22,0,0,0,0,0,0,9,0,0,0,0,0,0,3,0
9,6,0,12,0,25,0,0,0,0,0,0,10,0,0,0,0,0,0,3,0


In [59]:
# Calculate the cumulative ice requirements up till each day

XW_up = XW_add.cumsum()
XW_up.head(30)

,050 to 054,050 to 131,050 to 491,050 to 492,050 to 520,051 to 054,051 to 131,051 to 491,051 to 492,051 to 520,130 to 054,130 to 131,130 to 491,130 to 492,130 to 520,490 to 054,490 to 131,490 to 491,490 to 492,490 to 520
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,0,11,0,22,0,0,0,0,0,0,9,0,0,0,0,0,0,3,0
2,11,0,23,0,47,0,0,0,0,0,0,19,0,0,0,0,0,0,6,0
3,16,0,32,0,66,0,0,0,0,0,0,27,0,0,0,0,0,0,8,0
4,18,0,36,0,76,0,0,0,0,0,0,31,0,0,0,0,0,0,9,0
5,24,0,49,0,103,0,0,0,0,0,0,42,0,0,0,0,0,0,12,0
6,30,0,62,0,129,0,0,0,0,0,0,53,0,0,0,0,0,0,15,0
7,35,0,73,0,151,0,0,0,0,0,0,62,0,0,0,0,0,0,18,0
8,40,0,83,0,173,0,0,0,0,0,0,71,0,0,0,0,0,0,21,0
9,46,0,95,0,198,0,0,0,0,0,0,81,0,0,0,0,0,0,24,0


In [60]:
# Check which rows are not the same. Ideally, all rows should be same after doing .diff() on a .cumsum()
np.where(XW_up.diff() != XW_add)
# Or
XW_add.index[np.any(XW_up.diff() != XW_add,axis=1)]

Int64Index([0], dtype='int64')

In [61]:
# Round up cumulative ice requirement to multiples of 25
XW_25 = np.ceil((XW_up/25)) * 25

# Perform inverse cumulative sum to get days when rounded up cumulative requirement increased by 25. (Like getting pmf from cdf )
XW_25 = XW_25.diff().fillna(0).astype(int)

# Compare the following with raw schedule to verify if rounding up achieves the purpose of sending ice in multiples of 25 when previous surplus is insufficient.
XW_25.head(30).style.background_gradient(cmap='Blues')

,050 to 054,050 to 131,050 to 491,050 to 492,050 to 520,051 to 054,051 to 131,051 to 491,051 to 492,051 to 520,130 to 054,130 to 131,130 to 491,130 to 492,130 to 520,490 to 054,490 to 131,490 to 491,490 to 492,490 to 520
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,25,0,25,0,25,0,0,0,0,0,0,25,0,0,0,0,0,0,25,0
2,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,25,0,25,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0
4,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,25,0,25,0,25,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0
7,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,25,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0


In [0]:
# Change column names to be meaningful:

plant_code = facility_df[(facility_df['div_type_new']=="PLANT") & (facility_df['pod']==int(pod))][['div_number','index','pod']].sort_values('index',ascending=True)
setcolnames= ['from %s to oss' %(p) for p in plant_code.div_number ]
TOT_PS.columns=setcolnames

In [63]:
# Apply the same operations to PS:

TOT_PS.head(30).style.background_gradient(cmap='Blues')

,from 050 to oss,from 051 to oss,from 130 to oss,from 490 to oss
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [64]:
# Calculate the cumulative ice requirements up till each day

PS_up = TOT_PS.cumsum()
PS_up.head(30)

,from 050 to oss,from 051 to oss,from 130 to oss,from 490 to oss
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [65]:
# Check which rows are not the same. Ideally, all rows should be same after doing .diff() on a .cumsum()
np.where(PS_up.diff() != TOT_PS)
# Or
TOT_PS.index[np.any(PS_up.diff() != TOT_PS,axis=1)]

Int64Index([0], dtype='int64')

In [66]:
# Round up cumulative ice requirement to multiples of 25
PS_25 = np.ceil((PS_up/25)) * 25

# Perform inverse cumulative sum to get days when rounded up cumulative requirement increased by 25. (Like getting pmf from cdf )
PS_25 = PS_25.diff().fillna(0).astype(int)

# Compare the following with raw schedule to verify if rounding up achieves the purpose of sending ice in multiples of 25 when previous surplus is insufficient.
PS_25.head(30).style.background_gradient(cmap='Blues')

,from 050 to oss,from 051 to oss,from 130 to oss,from 490 to oss
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [67]:
pip install --user XlsxWriter

     |████████████████████████████████| 143kB 2.8MB/s 


In [0]:

# Change column names to be meaningful:

plant_code = facility_df[(facility_df['div_type_new']=="PLANT") & (facility_df['pod']==int(pod))][['div_number','index','pod']].sort_values('index',ascending=True)
setcolnames= ['%s ' %(p) for p in plant_code.div_number  ]
df_prod.columns=setcolnames

In [0]:
# Change column names to be meaningful:

plant_code = facility_df[(facility_df['div_type_new']=="PLANT") & (facility_df['pod']==int(pod))][['div_number','index','pod']].sort_values('index',ascending=True)
setcolnames= ['%s ' %(p) for p in plant_code.div_number  ]
df_shifts.columns=setcolnames

In [0]:
import xlsxwriter

In [0]:
with pd.ExcelWriter('Schedule.xlsx') as writer:  
    PS_25.to_excel(writer, sheet_name='Plant-OSS transport')
    XW_25.to_excel(writer, sheet_name='Plant-Terminal transport')
    XW_add.to_excel(writer,sheet_name='Plant-Terminal before transport')
    TOT_PS.to_excel(writer,sheet_name='Plant-OSS before transport')
    df_O.to_excel(writer,sheet_name='Production capacity')
    df_M.to_excel(writer,sheet_name='Plants storage')
    df_N.to_excel(writer,sheet_name='Terminals storage')
    df_term_inv.to_excel(writer, sheet_name="term_inv")
    df_plant_inv.to_excel(writer, sheet_name="plant_inv")
    df_shifts.to_excel(writer, sheet_name="shifts")
    df_prod.to_excel(writer, sheet_name="production")
    df_OSS_TOT.to_excel(writer, sheet_name="OSS usage")



In [0]:
from google.colab import files
files.download('Schedule.xlsx')

MessageError: ignored

In [0]:
days = plant_demand.Date.values[1:]
total_demand = np.roll(total_demand, 1)

In [74]:
#!pip install plotly==4.5.2

import plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=days, y=total_inv,
                    mode='lines',
                    name='Inventory(P+T)',
                    marker=dict(
                        color="rgb(0,50,0)"
                    ),
                    opacity= 0.5    ))
fig.add_trace(go.Scatter(x=days, y=total_production,
                    mode='lines',
                    marker=dict(
                        color="rgb(0,0,50)"
                    ),
                    opacity=0.3,
                    name='Production'))
fig.add_trace(go.Scatter(x=days, y=total_demand,
                    mode='lines',
                    name='Demand',
                    marker=dict(
                        color="rgb(200,0,0)"
                    ),
                    opacity = 0.3))
fig.add_trace(go.Scatter(x=days, y=total_oss,
                    mode='lines',
                    name='OSS',
                    marker=dict(
                        color="rgb(200,200,0)"
                    ),
                    opacity = 1))
fig.update_layout(title='Production Plan', title_x=0.5,
                   xaxis_title='Days',
                   yaxis_title='Number of Pallets')
fig.show()

In [0]:
Facility_list = list(df_model["div_name"])
Facility_list

In [0]:
#@ Select the name of the Facility to view
#Facility =  'GALLIPOLIS'#@param Facility_list
Facility = "MILTON" #@param ['MILTON', 'BANNER', 'MORGANTOWN', 'PARKERSBURG', 'PORTSMOUTH', 'CLARKSBURG', 'GALLIPOLIS', 'SUTTON', 'BECKLEY']

In [0]:
 fac_df = df_model[df_model["div_name"]==Facility]
 fac_type=fac_df["div_type_new"]
 fac_index=fac_df['index']
 fac_index=fac_index.values[0]
 fac_num=fac_df["div_number"].values[0]
 
if fac_type.values[0] =='PLANT':
  fac_prod=list()
  fac_inv=list()
  fac_deficit=list()
  fac_truck_deficit=list()
  fac_dem=list(b[fac_num])
  for day in days:
    fac_prod.append(P[(fac_index,day)].solution_value())
    fac_inv.append(m[(fac_index,day)].solution_value())
    fac_deficit.append(SP[(1,fac_index,day)].solution_value())
    fac_truck_deficit.append(Tr[(fac_index,day)].solution_value())
else:
  fac_inv=list()
  fac_deficit=list()
  fac_dem=list(d[fac_num])
  for day in days:
    fac_inv.append(n[(fac_index,day)].solution_value())
    fac_deficit.append(ST[(1,fac_index,day)].solution_value())    

In [0]:
if fac_type.values[0] =='PLANT':
  fig1 = go.Figure()
  fig1.add_trace(go.Scatter(x=days, y=fac_inv,
                    mode='lines',
                    name='Inventory',
                    marker=dict(
                        color="rgb(0,50,0)"
                    ),
                    opacity= 0.5    ))
  fig1.add_trace(go.Scatter(x=days, y=fac_prod,
                    mode='lines',
                    marker=dict(
                        color="rgb(0,0,50)"
                    ),
                    opacity=0.3,
                    name='Production'))
  fig1.add_trace(go.Scatter(x=days, y=fac_dem,
                    mode='lines',
                    name='Demand',
                    marker=dict(
                        color="rgb(200,0,0)"
                    ),
                    opacity = 0.3))
  fig1.add_trace(go.Scatter(x=days, y=fac_deficit,
                    mode='lines',
                    name='Deficit',
                    marker=dict(
                        color="rgb(200,200,0)"
                    ),
                    opacity = 1))
  fig1.update_layout(title='Production Plan of plant', title_x=0.5,
                   xaxis_title='Days',
                   yaxis_title='Number of Pallets')
  fig1.show()
else:
  fig1 = go.Figure()
  fig1.add_trace(go.Scatter(x=days, y=fac_inv,
                    mode='lines',
                    name='Inventory',
                    marker=dict(
                        color="rgb(0,50,0)"
                    ),
                    opacity= 0.5    ))
  fig1.add_trace(go.Scatter(x=days, y=fac_dem,
                    mode='lines',
                    name='Demand',
                    marker=dict(
                        color="rgb(200,0,0)"
                    ),
                    opacity = 0.3))
  fig1.add_trace(go.Scatter(x=days, y=fac_deficit,
                    mode='lines',
                    name='Deficit',
                    marker=dict(
                        color="rgb(200,200,0)"
                    ),
                    opacity = 1))
  fig1.update_layout(title='Inventory Plan of Terminal', title_x=0.5,
                   xaxis_title='Days',
                   yaxis_title='Number of Pallets')
  fig1.show()



In [0]:
##Plant Truck deficit

if fac_type.values[0] =='PLANT':
  fig1 = go.Figure()
  fig1.add_trace(go.Scatter(x=days, y=fac_truck_deficit,
                    mode='lines',
                    name='truck deficit',
                    marker=dict(
                        color="rgb(0,50,0)"
                    ),
                    opacity= 0.5    ))
  fig1.update_layout(title='Truck Deficit of plant', title_x=0.5,
                   xaxis_title='Days',
                   yaxis_title='Number of Trucks')
  fig1.show()

## Backup Code


production1[0] = 0
production2[0] = 0
production1[-2] = 0
production1[-1] = 0
production2[-2] = 0
production2[-1] = 0
plant_inv1[-1] = 0
plant_inv1[-2] = 0
plant_inv2[-1] = 0
plant_inv2[-2] = 0
plt.plot(production1)
plt.show()
plt.plot(production2)
plt.show()
plt.plot(plant_inv1)
plt.show()
plt.plot(plant_inv2)
plt.show()

plt.plot(production1)
plt.show()
plt.plot(production2)
plt.show()

plt.plot(plant_inv1)
plt.show()
plt.plot(plant_inv2)
plt.show()

import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.plot(days,total_inv,"r",label='Inventory')
plt.plot(days,total_production, "b",label='Production')
plt.plot(days, total_demand, "y",label='Demand')
plt.axhline(y=total_min_inv, linestyle="--",color="lightgrey")
plt.axhline(y=0, linestyle="--",color="lightgrey")
plt.xlim([1,num_days])
plt.legend(prop={"size":20})
plt.show()

input_file=pd.read_excel("clusters.xlsx")
input_file.head()


Plants=input_file[input_file['DivisionType']=='PLANT']
Terminals=input_file[input_file['DivisionType']=='TERM']

O=Plants['OnSiteProductionCapSkids']
M=Plants['OnSiteStorageCapSkids']+Plants['FleetPalletCount']
N=Terminals['OnSiteStorageCapSkids']+Terminals['FleetPalletCount']

P= {}
for plant in plants:
    for day in days:
        P[(plant, day)] = model.NumVar(0,100000,'P_%i,%i' % (plant, day))

X= {}
for plant in plants:
    for terminal in terminals:
        for day in days:
            X[(plant,terminal, day)] = model.NumVar(0,100000,'X_%i,%i,%i' % (plant,terminal, day))

Z= {}
for plant in plants:
    for day in days:
        Z[(plant, day)] = model.IntVar(0,3,'Z_%i,%i' % (plant, day))

Y= {}
for plant in plants:
    for day in days:
        Y[(plant, day)] = model.NumVar(0,100000,'Y_%i,%i' % (plant, day))

m= {}
for plant in plants:
    for day in days:
        m[(plant, day)] = model.NumVar(0,100000,'m_%i,%i' % (plant, day))

B= {}
for plant in plants:
    for day in days:
        B[(plant, day)] = model.NumVar(0,100000,'B_%i,%i' % (plant, day))

n= {}
for terminal in terminals:
    for day in days:
        n[(terminal, day)] = model.NumVar(0,100000,'n_%i,%i' % (terminal, day))

import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.plot(days,total_inv,"r",label='Inventory')
plt.plot(days,total_production, "b",label='Production')
plt.plot(days, total_demand, "y",label='Demand')
plt.axhline(y=total_min_inv, linestyle="--",color="lightgrey")
plt.axhline(y=0, linestyle="--",color="lightgrey")
plt.xlim([1,num_days])
plt.legend(prop={"size":20})
plt.show()

production_cost = sum( C[plant-1] * P[(plant,day)] for plant in plants for day in days)
transportation_cost = sum(T[plant-1][terminal-1] * X[(plant,terminal, day)]  for plant in plants for terminal in terminals for day in days)

model.Minimize(production_cost+transportation_cost)

solver = cp_model.CpSolver()
solver.Solve(model)
for day in days:
    print('Day', day)
    for plant in plants:
        print(solver.Value(P[(plant, day)]))
        print(solver.Value(m[(plant,day)]))